In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import docx
from string import punctuation
import re
import csv 
import os
from simhash import Simhash, SimhashIndex

In [3]:
from src.utils import search_API, fetch_details, create_csv_path, collectFromEndnote
# , writeFromEndnoteTOCsv

In [4]:
rba_path = 'data/1. Risk-benefit analysis Kinderformularium Oxycodone.docx'

In [5]:
csv_path = create_csv_path(rba_path)

In [6]:
referenceList = collectFromEndnote(rba_path)

In [7]:
def writeFromEndnoteTOCsv(referenceList, csv_path):
    header = ['record_id', 'title', 'abstract', 'doi', 'final_included']
    with open(csv_path, 'w', newline='') as csvfile:
        cw = csv.writer(csvfile, delimiter=',')
        cw.writerow(header)

    #Loop over the reference list in the docx file
    for ref in referenceList:
        search_query = ref['p_doi'] if ref['p_doi'] else ref['p_title'] 
    #     print('#search_query', search_query)
        try:
            PubmedSearchResults = search_API(search_query)
            id_list = PubmedSearchResults['IdList']
            #todo If id_list is empty find a ways to reproduce it
            papers = fetch_details(id_list)
            referenceFound = False

            for paperIndex, paper in enumerate(papers['PubmedArticle']):
                paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
                if (Simhash(paperTitle).distance(Simhash(ref['p_title'])) <= 5):
                    # Todo Add Author confirmation as well
                    doi = ''
                    for idn, ids in enumerate(papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList']):
                        if papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn].attributes['IdType'] == 'doi':
                            doi = papers['PubmedArticle'][paperIndex]['PubmedData']['ArticleIdList'][idn]

                    abstract = ''
                    if 'Abstract' in paper['MedlineCitation']['Article']:
                        abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])

                    data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                    with open(csv_path, 'a') as csvfile:
                        cw = csv.writer(csvfile, delimiter=',')
                        cw.writerow(data)
                    referenceFound = True 
                    break
            if not referenceFound:
                if (len(search_query.split(' ')) < 8):
                    search_query = search_query + ' ' + ref['p_authors'].split(' ')[0]
#                     print(search_query)
    #                 elif (len(search_query.split(' ')) >= 8):
    #                 If it is longer divide it half first then remove 1 by 1


        except:
            print('!!!! The reference could not be found automaticaly: ', ref['p_title'])


In [8]:
writeFromEndnoteTOCsv(referenceList, csv_path)